# Comments on the Examples

We have seen two sets of data that give pH rate prfiles from work dony in Tony Kirby's group at cambridge. 

## Example 2

In the second example the author describe how they obtained the rate constants for each step in a consecutive reaction. 

the reaction scheme is shown below...

$$A \xrightarrow{k_1} B + C \\ 
B \xrightarrow{k_2} C$$

The total reaction is...

$$A \xrightarrow{k_{obs{}}} 2C $$

There are two possibilities.

1. The first step is rate-determining. The kinetics will be the same as if the reaction produced two products in a single step.
2. The second is rate-determining.The kinetics will follow that of a consecutive reaction. See notebook `6_PhysOrgExamples/6_1_start.ipynb` for some code that has already been written for consecutive reactions.

The author present integrated rate equations for the observed absorbance for each of the two cases above. For the first case they used the total reaction with $k_{obs}$. In the second case they presented a much more complicatde equation. Can you integrate these equations. I can't. Thats why I use *Python*.

## Symbolic Integration

For the first case we can treat it as a simple first-order reaction with the rate consant of $k_{obs}$.

*SymPy* can perform integration and differentiation operations on many expressions. Let us explore integrating the simple rate equations that you are familiar with.

# Integrating First Order Reaction for A

The basic first order reaction might be described as such...

$$ A \xrightarrow{k} 2C $$ 

The rate can be expressed as the disappearance of A over time and is equal to the rate constant multiplied by the concentration of reacatant at a given time point...

$$ -\frac{d}{dt}[A]_t = k[A]_t $$

We can state that the amount of reactant consumed at a give time point is $x$ and that the concentration of reactant is the initial concentration less this value.

$$ [A]_t = [A]_0 - x $$

Substitute this into the equation for rate and we get...

$$ -\frac{d}{d t}([A]_0 - x) = k([A]_0 - x) $$

...which can be simplified to give...

 $$   \frac{d x}{d t} = k([A]_0 - x) $$

In your first year calculus class you would solve this by collecting the differentials for $x$ and $t$ on either side of the equation and then integrating each side over the span of time.

$$ \frac{d x}{[A]_0 - x} = k\,dt $$
$$ \int_0^x \frac{d x}{[A]_0 - x} = \int_0^t k\,dt $$

# Using *SymPy* Substitutions
Here I will demonstrate the symbolic math features of *SymPy*. Here we start with the rate law and use *SymPy* to make the substitutions discussed above. We will start with...

$$ -\frac{d}{dt}[A]_t = k[A]_t $$

...and then substitute in the expression for $A_t$ in terms of $x$ using *SymPy*

$$ [A]_t = [A]_0 - x $$

We will the solve the differential that is prosuced by the substitution. Observe we get the same answer.

# Absorbance

he authors observed the reaction as absorbance at 256nm over time. The absorbance contributed by each species is give by Beers law (assuming a 1 cm path length, as always)

$$ Abs_A = \epsilon_A [A]$$

In [13]:
import sympy as sym

t = sym.symbols('t')     # create t as a 'symbol', not a variable
k = sym.symbols('k')     # create k as a 'symbol'
At = sym.symbols('A_t')  # create At as a 'symbol'
A0 = sym.symbols('A0')   # create A0 as a 'symbol'

xt = sym.Function('x_t')    # create x as a 'function', not a variable

lhs = sym.Derivative(-At, t) # Using Derivative function to get differential of A(t) w.r.t. t
                            # could not used At.diff(t) because At is not a function 
rhs = k*(At)

diffeq = sym.Eq(lhs, rhs)   # create a sympy equation
print("The initial differential equation is...")
display(diffeq)             # display that equation

diffeq = diffeq.subs({At: (A0 - xt(t))})

print("After substituting the differential equation is...")
display(diffeq)             # display that equation


print("The integrated equation starting at x=0 when t=0 is...")
res = sym.dsolve(diffeq, ics={xt(0): 0})     # Solve the differential equation. 
                                            # Initial condition is x(t) = 0 when t = 0
display(res)
#dir(diffeq)

The initial differential equation is...


Eq(Derivative(-A_t, t), A_t*k)

After substituting the differential equation is...


Eq(Derivative(-A0 + x_t(t), t), k*(A0 - x_t(t)))

The integrated equation starting at x=0 when t=0 is...


Eq(x_t(t), A0 - A0*exp(-k*t))

In [51]:
x = sym.symbols('x')     # create x as a 'symbol', not a variable or function

xt_rhs = res.rhs   # Get the expression for x(t)
print("The expression for x(t) is...")
display(xt_rhs)

eq1 = sym.Eq(At, A0-x)  # create an equation that defines A(t) in terms of A0 and x
print("The equation for A(t) in terms of x(t) is...")
display(eq1)

eq1 = eq1.subs(x, xt_rhs)  # xt is an expression for x. Substitute x for xt.
print("The equation for A(t) after substituting in the term for x(t)")
display(eq1)


The expression for x(t) is...


A0 - A0*exp(-k*t)

The equation for A(t) in terms of x(t) is...


Eq(A_t, A0 - x)

The equation for A(t) after substituting in the term for x(t)


Eq(A_t, A0*exp(-k*t))

# Integrating First Order Reaction for C

We could find the expression for C easily by just stating that $[C]_t = 2([A]_0-[A]_t)$. 


In [40]:
xt_rhs = res.rhs   # Get the expression for x(t)
print("The expression for x(t) is...")
display(xt_rhs)

eq2 = sym.Eq(Ct, 2*xt_rhs)  # create an equation that defines X(t) in terms of A0 and x
print("The equation for C(t) in terms of x(t) is...")
display(eq2)

The expression for x(t) is...


A0 - A0*exp(-k*t)

The equation for C(t) in terms of x(t) is...


Eq(C_t, 2*A0 - 2*A0*exp(-k*t))

## Absorbance

the authors followed the reaction by absorbance. The amount of $A$ is decreasing and the amount of $C$ is increasing. If the final aborbance is different than the initial (e.q. A and C have different extinction coefficients) then we should be able to get kinetic parameters from an equation that describes the change in absorbance vs. time. 

The total $Abs$ will be $Abs_A(t) + Abs_C(t)$

And we can say that...

$$Abs(t) = \epsilon_A\cdot A_t + \epsilon_C\cdot C_t$$

$$Abs(t) = \epsilon_A\cdot A_{0} e^{- k t} + \epsilon_C\cdot 2 A_{0}(1-e^{- k t})$$

When we begin, the initial absorbance is due to $[A]_0$ and so we can say that $Abs_0 = \epsilon_A\cdot A_{0}$ and so if...

$$A_{0} = \frac{Abs_0}{\epsilon_A}$$

...then we can say that

$$Abs(t) = \epsilon_A\cdot \frac{Abs_0}{\epsilon_A} e^{- k t} + \epsilon_C\cdot 2 \frac{Abs_0}{\epsilon_A}(1-e^{- k t})$$

and after we simplify we get the same equation as the authors present in their paper (eq. 2). It is a useful exercise to check the math in any paper. Printing errors are common, especially in typeset equations and figures that are often reinterpretted in the printing process (especially a risk in older papers.)

$$Abs(t) =  {Abs_0} e^{- k t} + \frac{\epsilon_C}{\epsilon_A} 2 Abs_0(1-e^{- k t})$$

Now let us do all of that with *Python*


In [70]:
display(eq1)
display(eq2)

At_ = eq1.rhs
Ct_ = eq2.rhs

e_A = sym.symbols(r'\epsilon_A') 
e_C = sym.symbols(r'\epsilon_C') 

Abs_A = sym.symbols(r'Abs_A')
Abs_C = sym.symbols(r'Abs_C')
Abs_0 = sym.symbols('Abs_0')

Total = e_A*At_ + e_C*Ct_
Total = Total.subs(A0, Abs_0/e_A)
display(Total)

Total = sym.collect(Total, e_A)
display(Total)

Total = sym.collect(Total,2*Abs_0)
display(Total)


Eq(A_t, A0*exp(-k*t))

Eq(C_t, 2*A0 - 2*A0*exp(-k*t))

Abs_0*exp(-k*t) + \epsilon_C*(2*Abs_0/\epsilon_A - 2*Abs_0*exp(-k*t)/\epsilon_A)

Abs_0*exp(-k*t) + \epsilon_C*(2*Abs_0 - 2*Abs_0*exp(-k*t))/\epsilon_A

Abs_0*exp(-k*t) + 2*Abs_0*\epsilon_C*(1 - exp(-k*t))/\epsilon_A

In [83]:
from sympy import *

t = Symbol('t')
x10 = Symbol('A0')
k1, k2, k3 = symbols('k1 k2 k3')

A = Matrix([[-k1,  0,  0, 0],
            [ k1,-k2,  0, 0],
            [  0, k2,-k3, 0],
            [  0,  0, k3, 0]])


i_conc = Matrix([x10,0,0,0])

V, D = A.diagonalize()


print("V = ")
display(V)
print("D = ")
display(D)

print("x (t) = ")
display(simplify(V * diag(1, exp(-k1*t),exp(-k2*t),exp(-k3*t)) * V**-1 * i_conc))



V = 


Matrix([
[0, -k1**2/(k2*k3) + k1/k3 + k1/k2 - 1,            0,  0],
[0,               k1*(k1 - k3)/(k2*k3), (k2 - k3)/k3,  0],
[0,                             -k1/k3,       -k2/k3, -1],
[1,                                  1,            1,  1]])

D = 


Matrix([
[0,   0,   0,   0],
[0, -k1,   0,   0],
[0,   0, -k2,   0],
[0,   0,   0, -k3]])

x (t) = 


Matrix([
[                                                                                                                                                                                                                        A0*exp(-k1*t)],
[                                                                                                                                                                            A0*k1*(exp(k1*t) - exp(k2*t))*exp(-t*(k1 + k2))/(k1 - k2)],
[A0*k1*k2*(k1*exp(t*(k1 + k2)) - k1*exp(t*(k1 + k3)) - k2*exp(t*(k1 + k2)) + k2*exp(t*(k2 + k3)) + k3*exp(t*(k1 + k3)) - k3*exp(t*(k2 + k3)))*exp(-t*(k1 + k2 + k3))/(k1**2*k2 - k1**2*k3 - k1*k2**2 + k1*k3**2 + k2**2*k3 - k2*k3**2)],
[                                                                     -A0*(k1*k2*exp(-k3*t)/(k1*k2 - k1*k3 - k2*k3 + k3**2) - k1*k3*exp(-k2*t)/(k1*k2 - k1*k3 - k2**2 + k2*k3) + k2*k3*exp(-k1*t)/(k1**2 - k1*k2 - k1*k3 + k2*k3) - 1)]])

In [88]:
from sympy import *

t = Symbol('t')
x10 = Symbol('A0')
k1, k2 = symbols('k1 k2')

A = Matrix([[-k1,  0,  0],
            [ k1,-k2,  0],
            [  0, k2,  0]])


i_conc = Matrix([x10,0,0])

V, D = A.diagonalize()

print("conc = ")
display(i_conc)

print("V = ")
display(V)
print("D = ")
display(D)

print("x (t) = ")
display(simplify(V * diag(1, exp(-k1*t),exp(-k2*t)) * V**-1 * i_conc))



conc = 


Matrix([
[A0],
[ 0],
[ 0]])

V = 


Matrix([
[0, (k1 - k2)/k2,  0],
[0,       -k1/k2, -1],
[1,            1,  1]])

D = 


Matrix([
[0,   0,   0],
[0, -k1,   0],
[0,   0, -k2]])

x (t) = 


Matrix([
[                                                                             A0*exp(-k1*t)],
[                                   A0*k1*exp(-k2*t)/(k1 - k2) - A0*k1*exp(-k1*t)/(k1 - k2)],
[A0*(-k1*exp(k1*t) + k2*exp(k2*t) + (k1 - k2)*exp(t*(k1 + k2)))*exp(-t*(k1 + k2))/(k1 - k2)]])